In [77]:
import numpy as np
from scipy.integrate import simps
from scipy.interpolate import interp1d
from astropy import constants as const
import astropy.units as u
from Opacity_Tool_Profiles import Opacity_Tool_Profiles

In [95]:
class Disk_SED_Model:
    # This class is used to generate Disk Models using a radiative transfer model were emmission is isothermal
    # 
    #
    #
    
    def __init__(self, Dist):
        # Initialize constants use by Planck's Blackbody Radiation Function
        # Constants from the Use of Astropy with respective units removed.
        # Only one initializer input required: Distance to target.
        
        
        self.D = Dist * const.pc.cgs/u.cm            # Distance
        self.k_B = const.k_B.cgs/(u.erg/u.K)         # Boltzman constant
        self.h = const.h.cgs/(u.erg*u.s)             # Planck constant
        self.c = const.c.cgs/(u.cm/u.s)              # speed of light
        self.PI = np.pi
        
        self.Planck_scale_factor = 2*self.h*self.c      # Simplified Planck Funciton multi. factor
        self.Planck_exp_factor = self.h*self.c/self.k_B # Simplified Planck Function exp. multi. factor
        
        

    def Temperature_Function(self, Arb_Temperature, Arb_Radius, Radius_Grid, q):
"""        # Temperture Function as a power law with units in Kelvin [K]
           # This function takes in inputs: Arbitrary Temperature "T" (Normalization constants)
           #                                Arbitrary Radius "R"  (Normalization constants)
           #                                Radii grid "r" (single value or array)
           #                                Power Law index "q"
           #
           #
           # This fucntion returns a value/array of Temperatures at some radius or radii
 """       
        
        return Arb_Temperature*(r/Arb_Radius)**(-q)

    
    
    def Surface_Density_Funcition(self, Arb_Surface_Density, Arb_Radius, Radius_Grid, p):
"""        # Surface Density Function as a power law with units in cgs [g cm^-2]
           # This function takes in inputs: Arbitrary Surface Density "\Sigma_0" (Normalization constants)
           #                                Arbitrary Radius "R"  (Normalization constants)
           #                                Radius grid (single value or array)
           #                                Power Law index "p"
           #
           #
           # This fucntion returns a value/array of Surface Density at some radius or radii
 """       
        
        return Arb_Surface_Density*(Radius_Grid/Arb_Radius)**(-p)

    def Planck_Function(self, Wavelength_Grid,Temperature_Grid):
"""        # Planck Blackbody Function with units in cgs [ergs cm^-2 sterad^-1].
           # Modified version: lambda*nu = c      =>      nu = c/lambda 
           # This function takes in inputs: Wavelength Grid
           #                                Temperature Grid
           #
           #
           # This fucntion return value/array of Blackbody Radiation value at some wavelength(s)
 """             
            
            
        return (self.Planck_scale_factor/Wavelength_Grid**3)/(np.exp(self.Planck_exp_factor/(wavelength*Temperature_Grid))-1) 
    
    
    
    
    
   
    def Disk_Model(self, Data_Grid, Inner_Radius, Outer_Radius, Inclination, Arb_Radius, Arb_Temperature, 
                   Arb_Surface_Density, Q, P, amax, apow):
"""        # Disk Model Function that produces SED of a disk around YSO.
           # This Funcition has been modified to account for Opacity Profiles from DIANA PROJECT OPACITY TOOL
           # Reference(s): Ribas et al. 2016
           # This function takes in inputs: Data Grid
           #                                Inner Radius Value                  "R_in"
           #                                Outer Radius Value                  "R_out"
           #                                Inclination                         "Inc"
           #                                Arbitrary Radius                    "R_0"  
           #                                Arbitrary Temperature               "T_0" 
           #                                Arbitrary Surface Density           "\Sigma_0" 
           #                                Radius grid (single value or array)
           #                                Power Law Index                     "q"
           #                                Power Law Index                     "p"
           #                                Grain size max. (microns)           "amax"
           #                                Dust size powerlaw                  "apow"
           #
           #
           # This fucntion returns a value/array of Surface Density at some radius or radii
 """   
        
        # Define Radius grid using log spacing
        Radius_Grid = np.geomspace(R_In, R_Out, 50, endpoint=True) 
        
        # Convert Data from microns to cm ( must be in cgs)
        Wavelength_Grid = Data_Lambda_Grid * 1e-4 
        
        # Compute Circumference for intergral 2*pi*r
        Circumference = 2*np.pi*Radius_Grid * const.au.cgs/u.cm
        
        # Compute cosine and secant terms due to the incliniation of disk
        # Covert degrees to radians
        cos_i = np.cos((Inc * np.pi / 180))
        sec_i = 1/cos_i
        
        # Opacity_Tool_Profiles class must be initialized
        # Determine Opacity Profile given amax and apow
        # Opacity Grid of preloaded file
        Opacity_grid = Opacity_Tool.Opacity_Profile(amax,apow)
        
        # Interpolate Opacity_Grid 
        Interp_Opacity_grid = interp1d(Opacity_grid[0,:,0], Opacity_grid[0,:,1])
        
        # Interpolate Opacity Grid onto Wavelength_Grid
        Opacity_grid_onto_Data = Interp_Opacity_grid(Data_Lambda_Grid)
        
        # Initialize zero array with same dimension as Wavelength_Grid
        F_nu_cgs = np.zeros(len(Wavelength_Grid))
        
        # For loops to perform flux density computations
        for ii in range(len(Wavelength_Grid)):
            
            # Define Optical Depth grid
            Tau_Grid = self.Sig_r(Sigma,R,Radius_Grid,P)*Opacity_grid_onto_Data[ii]
            
            # Define Integrand
            Integrand = self.B_r(Wavelength_Grid[ii],self.T_r(Temp,R, Radius_Grid,Q))*((1-np.exp(-Tau_Grid*sec_i))*Circumference)
            
            
            # Evaluate Integral using simpson method
            Integral = simps(Integrand, Circumference/(2*self.PI))
            
            # Compute Flux density in cgs
            Flux_nu = (cos_i/self.D**2)*Integral
            
            # Add calucated flux denisty value at wavelength  ii to the iith position in empty array
            F_nu_cgs[ii] = Flux_nu
        
        # Convert cgs units to Jansky
        F_nu_Jansky = 1e23 * F_nu_cgs
        
        return F_nu_Jansky
